### 여름학기 csv

1. pdf -> csv

In [66]:
import pdfplumber
import pandas as pd

# PDF 파일 경로
pdf_file_path = r"./libs/pdf/2024학년도 하계 계절수업 교과목 개설 현황표_공지용.pdf"

# 저장할 CSV 파일 경로 .\libs\csv
#csv_file_path = ".\libs\csv/kwu-24summer-timetable.csv"

# 데이터를 저장할 리스트 초기화
data = []

# PDF 파일 열기
with pdfplumber.open(pdf_file_path) as pdf:
    # 각 페이지 순회
    for page in pdf.pages:
        # 페이지에서 표 추출
        table = page.extract_table()
        if table:
            # 추출된 표 데이터를 리스트에 추가
            data.extend(table)

# 데이터가 없는 빈 행을 필터링
filtered_data = [row for row in data if any(cell.strip() for cell in row)]

# 추출된 데이터를 DataFrame으로 변환
df = pd.DataFrame(filtered_data[1:], columns=filtered_data[0])  # 첫 번째 행을 헤더로 사용

# DataFrame을 CSV 파일로 저장
#df.to_csv(csv_file_path, index=False)

#print("CSV 파일이 저장되었습니다.")


In [67]:
print(df)

    순번              학과/교양영역  구분                   과목명 학점 개설\n강좌수         강의시간  \
0    1             전체공통(정보)  교필                C프로그래밍  3       1        1,2교시   
1    2  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3       1          15일   
2    3  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3       1        3,4교시   
3    4  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3       1        3,4교시   
4    5          전체공통(예술과체육)  교선               대중음악의역사  3       1          15일   
..  ..                  ...  ..                   ... ..     ...          ...   
56  57                 전체공통  교선          전략적의사결정과문제해결  3       1    광운대학교 박종구   
57  58                 전체공통  교선               행복한금융생활  3       1    광운대학교 송영출   
58  59                 전체공통  교선     화폐와금융의과거,현재,그리고미래  3       1    한양대학교 현정환   
59  60                 전체공통  교선               음식과세계문화  3       1  덕성여자대학교 김건희   
60  61                 전체공통  교선               그린IT의이해  3       1    한성대학교 조세홍   

      직종  담당교수             

2. 칼럼 '연도', '학기' 칼럼 추가
3. '개설강좌수', '직종' 칼럼 삭제

In [68]:
# '연도'와 '학기' 칼럼 추가
df.insert(1, '연도', '24')  # '순번' 뒤에 '연도' 칼럼 추가
df.insert(2, '학기', 'summer')  # '연도' 뒤에 '학기' 칼럼 추가

# '개설강좌수'와 '직종' 칼럼 삭제
df = df.drop(columns=['개설\n강좌수', '직종'])

# 결과 출력
print(df)

    순번  연도      학기              학과/교양영역  구분                   과목명 학점  \
0    1  24  summer             전체공통(정보)  교필                C프로그래밍  3   
1    2  24  summer  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3   
2    3  24  summer  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3   
3    4  24  summer  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3   
4    5  24  summer          전체공통(예술과체육)  교선               대중음악의역사  3   
..  ..  ..     ...                  ...  ..                   ... ..   
56  57  24  summer                 전체공통  교선          전략적의사결정과문제해결  3   
57  58  24  summer                 전체공통  교선               행복한금융생활  3   
58  59  24  summer                 전체공통  교선     화폐와금융의과거,현재,그리고미래  3   
59  60  24  summer                 전체공통  교선               음식과세계문화  3   
60  61  24  summer                 전체공통  교선               그린IT의이해  3   

           강의시간  담당교수               비 고  
0         1,2교시   정진범                    
1           15일   곡효운  원격강의 (사전녹화 100%)  
2        

In [69]:
df.columns

Index(['순번', '연도', '학기', '학과/교양영역', '구분', '과목명', '학점', '강의시간', '담당교수', '비 고'], dtype='object')

4. [수동확인 필요] '학과/교양영역' 칼럼 3개로 분리 (학과 : cantakeBunBan / 학과 : majorRecogBunBan), (교양영역 : lecTheme)

In [70]:
# '학과/교양영역' 칼럼을 세 개의 새로운 칼럼으로 분리
def split_major_lec(row):
    lecTheme = ''
    cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
    majorRecogBunBan = ''
    
    # 괄호 안의 내용을 교양영역으로 설정
    if '(' in row and ')' in row:
        lecTheme = row[row.find('(')+1:row.find(')')]

    # 전체공통에 따른 분리 !! [계절마다 추가해야할 수도 있음]
    if '전체공통' in row:
        cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
        majorRecogBunBan = ''
    else:
        cantakeBunBan = 'E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C2,C3,C4,C5,C6,C7,C8,C9,J5,J6,J7,J8,A1,A2,K1,K2,K3,N1,N2,N3,N4,N5,P1,B1,B2,B3,B4,B5,B6,R1,R2,R3,R4,R5,M1,M2,S1,S2,S3,L1,L2,L3,V1,V2,V3,V4'
        if '국제통상학부' in row:
            majorRecogBunBan = 'B5,B6'
        elif '법학부' in row:
            majorRecogBunBan = 'L1,L2,L3'
        elif '미디어커뮤니케이션학부' in row:
            majorRecogBunBan = 'M1,M2'
        elif '동북아문화산업학부' in row:
            majorRecogBunBan = 'R4,R5'
        elif '소프트웨어학부' in row:
            majorRecogBunBan = 'C4,C5,C6'
    
    return cantakeBunBan, majorRecogBunBan, lecTheme

df[['수강가능한 학과', '전공 인정 학과 ', '테마']] = df['학과/교양영역'].apply(lambda x: pd.Series(split_major_lec(x)))

df

,순번,연도,학기,학과/교양영역,구분,과목명,학점,강의시간,담당교수,비 고,수강가능한 학과,전공 인정 학과,테마
0,1,24,summer,전체공통(정보),교필,C프로그래밍,3,"1,2교시",정진범,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,정보
1,2,24,summer,전체공통\n(글로벌문화와제2외국어),교선,"초급중국어1\n(구,기초중국어회화1)",3,15일,곡효운,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
2,3,24,summer,전체공통\n(글로벌문화와제2외국어),교선,일본애니메이션과문화,3,"3,4교시",유양근,원격강의 (사전녹화 87%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
3,4,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급일본어1,3,"3,4교시",김희성,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어
4,5,24,summer,전체공통(예술과체육),교선,대중음악의역사,3,15일,김철수,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,57,24,summer,전체공통,교선,전략적의사결정과문제해결,3,광운대학교 박종구,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
57,58,24,summer,전체공통,교선,행복한금융생활,3,광운대학교 송영출,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
58,59,24,summer,전체공통,교선,"화폐와금융의과거,현재,그리고미래",3,한양대학교 현정환,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,
59,60,24,summer,전체공통,교선,음식과세계문화,3,덕성여자대학교 김건희,None,None,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,


5. [수동확인필요] 연계전공은 대학원을 연계해서 듣는 수업을 의미, 따라서 이수구분이 '연선', '연필' 과목 row는 삭제

In [71]:
# 필터링 전의 행 수
initial_count = len(df)

# '구분'이 '연선'이나 '연필'인 행을 제외하고 필터링
df_filtered = df[~df['구분'].isin(['연선', '연필'])]

# 필터링 후의 행 수
final_count = len(df_filtered)

# 삭제된 행의 수
deleted_count = initial_count - final_count

# 결과 출력
print(f"삭제된 행의 수: {deleted_count}")

삭제된 행의 수: 1


6. 강의시간표에 이러닝('대학교'가 있다면) 교수명으로 다 옮기기

In [72]:
df = df_filtered

# '담당교수' 칼럼이 None 또는 NaN일 경우 빈 문자열로 대체
df['담당교수'] = df['담당교수'].fillna('')

# '강의시간' 칼럼에 '대학교'가 포함된 행을 찾아서 '담당교수' 칼럼으로 텍스트를 옮기기
df['담당교수'] = df.apply(
    lambda row: row['담당교수'] + row['강의시간'] if '대학교' in row['강의시간'] else row['담당교수'],
    axis=1
)

# '강의시간' 칼럼에서 '대학교' 텍스트가 포함된 경우 전체 텍스트 삭제
df['강의시간'] = df['강의시간'].apply(
    lambda x: '' if '대학교' in x else x
)

# 결과 확인
print(df)

    순번  연도      학기              학과/교양영역  구분                   과목명 학점  \
0    1  24  summer             전체공통(정보)  교필                C프로그래밍  3   
1    2  24  summer  전체공통\n(글로벌문화와제2외국어)  교선  초급중국어1\n(구,기초중국어회화1)  3   
2    3  24  summer  전체공통\n(글로벌문화와제2외국어)  교선            일본애니메이션과문화  3   
3    4  24  summer  전체공통\n(글로벌문화와제2외국어)  교선                초급일본어1  3   
4    5  24  summer          전체공통(예술과체육)  교선               대중음악의역사  3   
5    6  24  summer          전체공통(예술과체육)  교선               생활속의디자인  3   
6    7  24  summer             전체공통(수학)  교선                 공학수학1  3   
7    8  24  summer             전체공통(수학)  교선                 공학수학1  3   
8    9  24  summer             전체공통(수학)  교선                 공학수학2  3   
9   10  24  summer             전체공통(수학)  교선                 공학수학2  3   
10  11  24  summer             전체공통(수학)  교선               기초수학및연습  3   
11  12  24  summer             전체공통(수학)  교선              대학수학및연습1  3   
12  13  24  summer             전체공통(수학)  교선              대학수학및연습

C:\Users\user\AppData\Local\Temp\ipykernel_11840\761209113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['담당교수'] = df['담당교수'].fillna('')
C:\Users\user\AppData\Local\Temp\ipykernel_11840\761209113.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['담당교수'] = df.apply(
C:\Users\user\AppData\Local\Temp\ipykernel_11840\761209113.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

7. '기타' 칼럼을 만들어서 구과목명에 대한 정보 담기
    새로운 칼럼인 '기타' 생성
    '(구,)' 텍스트가 있는 경우 기타로 '(' 칼럼부터 옮기기

In [73]:
# 기타 칼럼 생성
df['기타'] = df['과목명'].str.extract(r'(\(구,.*\))')

# 강의명 칼럼에서 기타 부분 제거
df['과목명'] = df['과목명'].str.replace(r'\(구,.*?\)', '', regex=True).str.strip()

df  

C:\Users\user\AppData\Local\Temp\ipykernel_11840\452461664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['기타'] = df['과목명'].str.extract(r'(\(구,.*\))')
C:\Users\user\AppData\Local\Temp\ipykernel_11840\452461664.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['과목명'] = df['과목명'].str.replace(r'\(구,.*?\)', '', regex=True).str.strip()


,순번,연도,학기,학과/교양영역,구분,과목명,학점,강의시간,담당교수,비 고,수강가능한 학과,전공 인정 학과,테마,기타
0,1,24,summer,전체공통(정보),교필,C프로그래밍,3,"1,2교시",정진범,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,정보,NaN
1,2,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급중국어1,3,15일,곡효운,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,"(구,기초중국어회화1)"
2,3,24,summer,전체공통\n(글로벌문화와제2외국어),교선,일본애니메이션과문화,3,"3,4교시",유양근,원격강의 (사전녹화 87%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,NaN
3,4,24,summer,전체공통\n(글로벌문화와제2외국어),교선,초급일본어1,3,"3,4교시",김희성,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,글로벌문화와제2외국어,NaN
4,5,24,summer,전체공통(예술과체육),교선,대중음악의역사,3,15일,김철수,원격강의 (사전녹화 100%),"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,NaN
5,6,24,summer,전체공통(예술과체육),교선,생활속의디자인,3,"4,5교시",김문석,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,예술과체육,NaN
6,7,24,summer,전체공통(수학),교선,공학수학1,3,"1,2교시",장정환,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,NaN
7,8,24,summer,전체공통(수학),교선,공학수학1,3,"5,6교시",최현옥,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,NaN
8,9,24,summer,전체공통(수학),교선,공학수학2,3,"3,4교시",윤미,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,NaN
9,10,24,summer,전체공통(수학),교선,공학수학2,3,"5,6교시",안소영,,"E1,E2,E3,E4,E5,E6,E7,E8,J1,J2,J3,J4,T1,T2,C1,C...",,수학,NaN


In [74]:
df_filtered.to_csv(".\libs\csv/kwu-24summer-timetable_v1.csv") # v1은 csv를 통해 만들어진 것을 의미 

8. 유의사항

    1. 학과/교양영역 -> 칼럼 검증을 위해 남겨둔거지 삭제해두됨
    
    2. 비고 -> 강의계획서로 채울거임(원격강의(%정도), e러닝 유무) 삭제해도됨

9. 강의계획서 크롤링 csv와 concat 하기
학정번호(lecNumber) 강의시간(lecTime)  강의시수(lecWeekTime)   강의실(lecClassroom)  비고(원격강의)  비고(e러닝 유무)

+ 이외에 강의계획서를 통해 뽑을 수 있는 모든 것들 

In [75]:
### kwu-24summer-timetable_v2 : csv 칼럼화 + 강의계획서 크롤링(***) 합치기 + 에타강의평은 학정번호 같은걸로 연결시키기 